<a href="https://colab.research.google.com/github/vincent4u/CE807_Text_Analytics/blob/main/Model_Dis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

In [69]:
import numpy as np
import os
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix



import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import lightgbm as lgb
from sklearn.multioutput import MultiOutputRegressor

from scipy.stats import pearsonr
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt


In [70]:
student_id = 2314555
np.random.seed(student_id)

In [71]:
train_df = pd.read_csv('/content/train.csv')
#train_df.head(20)

In [72]:
val_df=pd.read_csv('/content/valid.csv')
#val_df.head(20)

In [73]:
test_df=pd.read_csv('/content/test.csv')
#test_df.head(20)

In [74]:
print(train_df.columns)

Index(['comment_id', 'comment', 'split', 'toxicity'], dtype='object')


In [ ]:
print(test_df.columns)

In [ ]:
print(train_df['comment'])

# ***Text Pre-processing***

In [75]:

# Preprocessing steps
def preprocess_text(text):
    # Remove special tokens
    # Remove special tokens
    # Remove specific tokens
    text = text.replace("SDATA_5 :", "").replace("EDATA_5", "").replace("NEWLINE_TOKEN", "")

    # Remove special characters and punctuation marks using regular expressions
    text = re.sub(r'[^\w\s]', '', text)

    # Remove URLs, email addresses, and phone numbers
    text = re.sub(r"http\S+|www\S+|\S+@\S+|\d{10}", "", text)

    # Remove numbers
    text = re.sub(r"\d+", "", text)

    # Remove non-ASCII characters
    text = text.encode("ascii", "ignore").decode()

    # Convert the text to lowercase
    text = text.lower()

    # Remove extra white spaces
    text = re.sub(r'\s+', ' ', text)

    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Apply stemming or lemmatization
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]

    # Join the tokens back into a single string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

# Preprocess the text
train_df['preprocessed_comment'] = train_df['comment'].apply(preprocess_text)
val_df['preprocessed_comment'] = val_df['comment'].apply(preprocess_text)

#print(train_df['preprocessed_comment'])

0       kindergarden ignorantsv ad pakistani articl pa...
1                       violat rrv rule may ban wikipedia
2                          judg messag tp could use advic
3                                     love fatti sex mmmm
4               pleas could explain think warn page thank
                              ...                        
8694    utci beg differ histori huge import licens rea...
8695            peni sex grawp haeppenispenisvaginavagina
8696                         that exactli gamer die breed
8697                           articl unless fact support
8698                 milton friedman lp also measur thing
Name: preprocessed_comment, Length: 8699, dtype: object


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup

def preprocess_text(text):
    # Remove specific tokens
    text = text.replace("SDATA_5 :", "").replace("EDATA_5", "").replace("NEWLINE_TOKEN", "")


    # Remove special characters and punctuation marks using regular expressions
    text = re.sub(r'[^\w\s]', '', text)

    # Remove URLs, email addresses, and phone numbers
    text = re.sub(r"http\S+|www\S+|\S+@\S+|\d{10}", "", text)

    # Remove numbers
    text = re.sub(r"\d+", "", text)

    # Remove non-ASCII characters
    text = text.encode("ascii", "ignore").decode()

    # Convert the text to lowercase
    text = text.lower()

    # Remove extra white spaces
    text = re.sub(r"\s+", " ", text)

    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words("english"))
    tokens = [word for word in tokens if word not in stop_words]

    # Apply lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Join the tokens back into a single string
    preprocessed_text = " ".join(tokens)

    return preprocessed_text

# Preprocess the text
train_df['preprocessed_comment'] = train_df['comment'].apply(preprocess_text)
val_df['preprocessed_comment'] = val_df['comment'].apply(preprocess_text)

print(train_df['preprocessed_comment'])

In [ ]:
print(val_df['preprocessed_comment'])

# ***Building the Model***

***Model = LogisticRegression()***

In [64]:
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', LogisticRegression(max_iter=1000))
])

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'vectorizer__max_features': [1000, 5000, 10000],
    'classifier__C': [0.1, 1.0, 10.0]
}

# Perform grid search
grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=5)
grid_search.fit(train_df['preprocessed_comment'], train_df['toxicity'])

# Get the best model from grid search
best_model = grid_search.best_estimator_

# Testing the best model on validation data
val_pred = best_model.predict(val_df['preprocessed_comment'])

# Calculate metrics
accuracy = accuracy_score(val_df['toxicity'], val_pred)
recall = recall_score(val_df['toxicity'], val_pred, average='macro')
precision = precision_score(val_df['toxicity'], val_pred, average='macro')
f1 = f1_score(val_df['toxicity'], val_pred, average='macro')

# Print metrics
print('Accuracy:', accuracy)
print('Recall (macro):', recall)
print('Precision (macro):', precision)
print('F1 Score (macro):', f1)

# Compute and print confusion matrix
labels = np.unique(np.concatenate((val_df['toxicity'], val_pred)))
cm = confusion_matrix(val_df['toxicity'], val_pred, labels=labels)
print('Confusion Matrix:')
print(cm)

Accuracy: 0.8643835616438356
Recall (macro): 0.5018715182402274
Precision (macro): 0.5298707271801442
F1 Score (macro): 0.47347714903940724
Confusion Matrix:
[[2520   17]
 [ 379    4]]


***Model2 =X-Gboost***

In [95]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer


#Best hyperparameters: {'max_depth': 5, 'learning_rate': 0.029624377829993845, 'n_estimators': 649, 'subsample': 0.9815939695298614, 'colsample_bytree': 0.5082728972365318, 'gamma': 0.031807764002275224, 'min_child_weight': 8, 'reg_alpha': 0.30995407377084716, 'reg_lambda': 0.001118504465386989}

#Best hyperparameters: {'max_depth': 8, 'learning_rate': 0.0010171129315949034, 'n_estimators': 339, 'subsample': 0.7959607535620461, 'colsample_bytree': 0.7515029363148247}

# Define and train the model
# best_params = {
#     "objective": "reg:squarederror",
#     "n_estimators": 500,
#     "verbosity": 0,
#     "learning_rate": 0.0255709534112944,
#     "max_depth": 6,
#     "subsample":  0.8750699631919016,
#     "colsample_bytree": 0.41208987840060635,
#     "min_child_weight": 10
# }

best_params = {
    "objective": "reg:squarederror",
    "n_estimators": 649,
    "verbosity": 0,
    "learning_rate": 0.029624377829993845,
    "max_depth": 5,
    "subsample":  0.9815939695298614,
    "colsample_bytree": 0.5082728972365318,
    "min_child_weight": 8
}


# Create an XGBRegressor model with the best hyperparameters
vectorizer = TfidfVectorizer()

# Fit the vectorizer on the training data and transform the training data
train_features = vectorizer.fit_transform(train_df['preprocessed_comment'])

# Transform the validation data using the fitted vectorizer
val_features = vectorizer.transform(val_df['preprocessed_comment'])

# Create an instance of XGBClassifier with the best parameters
model = XGBClassifier(**best_params, tree_method='hist')

# Fit the model on the training features and labels
model.fit(train_features, train_df['toxicity'])

# Predict on the validation features
val_pred = model.predict(val_features)


# Calculate metrics
accuracy = accuracy_score(val_df['toxicity'], val_pred)
recall = recall_score(val_df['toxicity'], val_pred, average='macro')
precision = precision_score(val_df['toxicity'], val_pred, average='macro')
f1 = f1_score(val_df['toxicity'], val_pred, average='macro')

# Print metrics
print('Accuracy:', accuracy)
print('Recall (macro):', recall)
print('Precision (macro):', precision)
print('F1 Score (macro):', f1)

# Compute and print confusion matrix
labels = np.unique(np.concatenate((val_df['toxicity'], val_pred)))
cm = confusion_matrix(val_df['toxicity'], val_pred, labels=labels)
print('Confusion Matrix:')
print(cm)



Accuracy: 0.8681506849315068
Recall (macro): 0.5007142335214285
Precision (macro): 0.5594993141289437
F1 Score (macro): 0.46728231052200364
Confusion Matrix:
[[2534    3]
 [ 382    1]]


***Hyperparameter tuning for XGBoost***

In [80]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.4 MB/s eta 0:00:00


In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
import optuna

def objective(trial):
    # Split your data into training and validation sets
    params = {
        "objective": "binary:logistic",  # Use binary classification objective for XGBClassifier
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=10),
        "verbosity": 0,
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "max_depth": trial.suggest_int("max_depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 20),
    }

    # Create an instance of the TF-IDF vectorizer
    vectorizer = TfidfVectorizer()

    # Fit the vectorizer on the training data and transform the training data
    train_features = vectorizer.fit_transform(train_df['preprocessed_comment'])

    # Transform the validation data using the fitted vectorizer
    val_features = vectorizer.transform(val_df['preprocessed_comment'])

    # Create an XGBClassifier model with the best hyperparameters
    model = xgb.XGBClassifier(**params)
    model.fit(train_features, train_df['toxicity'], verbose=False)
    val_pred = model.predict(val_features)

    # Calculate metrics
    accuracy = accuracy_score(val_df['toxicity'], val_pred)
    recall = recall_score(val_df['toxicity'], val_pred, average='macro')
    precision = precision_score(val_df['toxicity'], val_pred, average='macro')
    f1 = f1_score(val_df['toxicity'], val_pred, average='macro')

    # Print metrics
    print('Accuracy:', accuracy)
    print('Recall (macro):', recall)
    print('Precision (macro):', precision)
    print('F1 Score (macro):', f1)

    # Compute and print confusion matrix
    labels = np.unique(np.concatenate((val_df['toxicity'], val_pred)))
    cm = confusion_matrix(val_df['toxicity'], val_pred, labels=labels)
    print('Confusion Matrix:')
    print(cm)

    return cm



study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=5)

if len(study.trials) > 0:
    print('Best hyperparameters:', study.best_params)
    print('Best Confusion Matrix:', study.best_value)
else:
    print('No trials are completed yet.')

In [90]:
import optuna
import xgboost as xgb
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix

def objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_loguniform('gamma', 0.01, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 0.001, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 0.001, 1.0),
    }

    model = xgb.XGBClassifier(**params)
    model.fit(train_features, train_df['toxicity'], verbose=False)
    val_pred = model.predict(val_features)

    accuracy = accuracy_score(val_df['toxicity'], val_pred)
    return -accuracy  # Optuna minimizes the objective

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

best_params = study.best_params
print('Best hyperparameters:', best_params)

model = xgb.XGBClassifier(**best_params)
model.fit(train_features, train_df['toxicity'], verbose=False)
val_pred = model.predict(val_features)

accuracy = accuracy_score(val_df['toxicity'], val_pred)
recall = recall_score(val_df['toxicity'], val_pred, average='macro')
precision = precision_score(val_df['toxicity'], val_pred, average='macro')
f1 = f1_score(val_df['toxicity'], val_pred, average='macro')

print('Accuracy:', accuracy)
print('Recall (macro):', recall)
print('Precision (macro):', precision)
print('F1 Score (macro):', f1)

labels = np.unique(np.concatenate((val_df['toxicity'], val_pred)))
cm = confusion_matrix(val_df['toxicity'], val_pred, labels=labels)
print('Confusion Matrix:')
print(cm)

[I 2024-04-09 19:34:35,310] A new study created in memory with name: no-name-3c9ca06c-7cde-4ecf-97d0-e4494f9a23bc
<ipython-input-90-e87bfbc25fac>:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
<ipython-input-90-e87bfbc25fac>:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
<ipython-input-90-e87bfbc25fac>:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0

Best hyperparameters: {'max_depth': 4, 'learning_rate': 0.044965824897193495, 'n_estimators': 157, 'subsample': 0.8363454732665425, 'colsample_bytree': 0.5487718104778876, 'gamma': 0.057977369562568916, 'min_child_weight': 6, 'reg_alpha': 0.0033561082497758107, 'reg_lambda': 0.003991605304480782}
Accuracy: 0.8688356164383562
Recall (macro): 0.5
Precision (macro): 0.4344178082191781
F1 Score (macro): 0.46490745831042696
Confusion Matrix:
[[2537    0]
 [ 383    0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [96]:
import optuna
import xgboost as xgb
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix

def objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
    }

    model = xgb.XGBClassifier(**params)
    model.fit(train_features, train_df['toxicity'], verbose=False)
    val_pred = model.predict(val_features)

    accuracy = accuracy_score(val_df['toxicity'], val_pred)
    return -accuracy  # Optuna minimizes the objective

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

best_params = study.best_params
print('Best hyperparameters:', best_params)

model = xgb.XGBClassifier(**best_params)
model.fit(train_features, train_df['toxicity'], verbose=False)
val_pred = model.predict(val_features)

accuracy = accuracy_score(val_df['toxicity'], val_pred)
recall = recall_score(val_df['toxicity'], val_pred, average='macro')
precision = precision_score(val_df['toxicity'], val_pred, average='macro')
f1 = f1_score(val_df['toxicity'], val_pred, average='macro')

print('Accuracy:', accuracy)
print('Recall (macro):', recall)
print('Precision (macro):', precision)
print('F1 Score (macro):', f1)

labels = np.unique(np.concatenate((val_df['toxicity'], val_pred)))
cm = confusion_matrix(val_df['toxicity'], val_pred, labels=labels)
print('Confusion Matrix:')
print(cm)

[I 2024-04-09 20:11:00,519] A new study created in memory with name: no-name-1ffe97a0-221f-4767-96a7-c7e93acb969e
<ipython-input-96-d737de303420>:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
<ipython-input-96-d737de303420>:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
<ipython-input-96-d737de303420>:11: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0

Best hyperparameters: {'max_depth': 8, 'learning_rate': 0.0010171129315949034, 'n_estimators': 339, 'subsample': 0.7959607535620461, 'colsample_bytree': 0.7515029363148247}
Accuracy: 0.8688356164383562
Recall (macro): 0.5
Precision (macro): 0.4344178082191781
F1 Score (macro): 0.46490745831042696
Confusion Matrix:
[[2537    0]
 [ 383    0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
